In [2]:
from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load("vectors_new_v2.kv", mmap='r')

In [3]:
import pandas as pd

In [4]:
netflix_id_to_movie_dict = pd.read_pickle("netflix_id_to_movie_dict.pkl")

In [5]:
def get_recommendations(users_likes, n = 5):
    #users_likes - list of netflix film ids as string
    #topn - num most likely
    word_vectors.most_similar_cosmul(positive=users_likes)
    all_recomendations = []
    for like in users_likes:
        all_recomendations += word_vectors.similar_by_word(like, topn=n)
    all_recomendations = list(filter(lambda x: x[0] not in users_likes ,list(set(list(all_recomendations)))))
    all_recomendations.sort(key = lambda x: x[1], reverse = True)
    all_recomendations_new = []
    used_ids = []
    for rec in all_recomendations:
        if rec[0] not in used_ids:
            all_recomendations_new.append(rec)
            used_ids.append(rec[0])
    result = []
    for recom in all_recomendations_new[:n]:
        result.append({"netflix_id": recom[0], "similarity": recom[1], "film_name": netflix_id_to_movie_dict[recom[0]]})
    return result

In [6]:
# 3524 - Pulp Fiction, 8072 - Seven (also Taranino's film)
get_recommendations(['3524', '8072'])

[{'netflix_id': '3303',
  'similarity': 0.8000013828277588,
  'film_name': 'Reservoir Dogs: Bonus Material'},
 {'netflix_id': '16586',
  'similarity': 0.7958824634552002,
  'film_name': "Jackie Brown: Collector's Series: Bonus Material"},
 {'netflix_id': '673',
  'similarity': 0.7902252078056335,
  'film_name': 'Fight Club: Bonus Material'},
 {'netflix_id': '2689',
  'similarity': 0.7735550403594971,
  'film_name': 'Minority Report: Bonus Material'},
 {'netflix_id': '4551',
  'similarity': 0.7523927688598633,
  'film_name': 'Snatch: Bonus Material'}]

In [52]:
netflix_id_2_ml100k_id = pd.read_pickle("netflix_id_2_ml100k_id.pkl")
ml100k_id_2_netflix_id = pd.read_pickle("ml100k_id_2_netflix_id.pkl")

In [33]:
len(ml100k_id_2_netflix_id)

1345

In [34]:
len(ml100k_id_2_netflix_id)

1345

In [71]:
# Most important
def get_recommendations_for_ml100k(users_likes, n = 100):
    #users_likes - list of ml100k film ids as string
    #topn - num most likely
    users_likes_netflix = [ml100k_id_2_netflix_id[x] for x in filter(lambda y: y in ml100k_id_2_netflix_id,users_likes)]
    if users_likes_netflix:
        result_netflix = get_recommendations(list(users_likes_netflix), n)
        result_ml_100k = []
        for res in result_netflix:
            if len(result_ml_100k) == 5:
                return result_ml_100k
            if res['netflix_id'] in netflix_id_2_ml100k_id:
                res['ml100k_id'] = str(netflix_id_2_ml100k_id[res['netflix_id']])
                result_ml_100k.append(res)
        return result_ml_100k
    return []

In [72]:
# '71' - 'Lion King, The (1994)'
get_recommendations_for_ml100k(['71'])

[{'netflix_id': '7989',
  'similarity': 0.4721536338329315,
  'film_name': 'Beauty and the Beast: Special Edition',
  'ml100k_id': '588'},
 {'netflix_id': '4127',
  'similarity': 0.4488835334777832,
  'film_name': 'The Flintstones',
  'ml100k_id': '383'},
 {'netflix_id': '17158',
  'similarity': 0.44245103001594543,
  'film_name': 'The Jungle Book',
  'ml100k_id': '465'},
 {'netflix_id': '14771',
  'similarity': 0.4338449239730835,
  'film_name': '32 Short Films About Glenn Gould',
  'ml100k_id': '811'},
 {'netflix_id': '16608',
  'similarity': 0.4115031957626343,
  'film_name': 'Buddy',
  'ml100k_id': '1608'}]

In [74]:
#96 - 'Terminator 2: Judgment Day (1991)'
get_recommendations_for_ml100k(['96'])

[]